In [1]:
import imageai
import tensorflow as tf
import cv2
import numpy as np
import pandas as pd


In [2]:
tf.config.run_functions_eagerly(True)


In [4]:
from imageai.Detection.Custom import DetectionModelTrainer
trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()

trainer.setDataDirectory("final_model")
trainer.setTrainConfig(object_names_array=['house', 'person', 'tree', 'flower', 'sun', 'knife', 'hammer', 'scissors',
                       'rocket', 'rifle', 'rainbow'], batch_size=10, num_experiments=1, train_from_pretrained_model="pretrained-yolov3.h5")
trainer.trainModel()


Generating anchor boxes for training images and annotation...
Average IOU for 9 anchors: 0.75
Anchor Boxes generated.
Detection configuration saved in  final_model\json\detection_config.json
Training on: 	['flower', 'hammer', 'house', 'knife', 'person', 'rainbow', 'rifle', 'rocket', 'scissors', 'sun', 'tree']
Training with Batch Size:  10
Number of Experiments:  1
Training with transfer learning from pretrained Model


C:\Users\Rania\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
C:\Users\Rania\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\training.py:1969: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


C:\Users\Rania\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\data\ops\dataset_ops.py:4211: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(
C:\Users\Rania\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


912/912 [==============================] - 31280s 34s/step - loss: 31.8822 - yolo_layer_loss: 5.2008 - yolo_layer_1_loss: 8.1949 - yolo_layer_2_loss: 18.4866 - val_loss: 13.9673 - val_yolo_layer_loss: 1.1831 - val_yolo_layer_1_loss: 4.2096 - val_yolo_layer_2_loss: 8.5746


In [5]:
from imageai.Detection.Custom import DetectionModelTrainer

trainer = DetectionModelTrainer()
trainer.setModelTypeAsYOLOv3()
trainer.setDataDirectory(data_directory="final_model")
metrics = trainer.evaluateModel(model_path="./final_model/models", json_path="./final_model/json/detection_config.json",
                                iou_threshold=0.2, object_threshold=0.2, nms_threshold=0.2)
print(metrics)


Starting Model evaluation....


C:\Users\Rania\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model File:  ./final_model/models\detection_model-ex-001--loss-0031.882.h5 

Using IoU :  0.2
Using Object Threshold :  0.2
Using Non-Maximum Suppression :  0.2
flower: 0.4009
hammer: 0.0000
house: 0.8123
knife: 0.0818
person: 0.7726
rainbow: 0.0000
rifle: 0.5921
rocket: 0.2459
scissors: 0.0000
sun: 0.7487
tree: 0.1388
mAP: 0.3448
[{'model_file': './final_model/models\\detection_model-ex-001--loss-0031.882.h5', 'using_iou': 0.2, 'using_object_threshold': 0.2, 'using_non_maximum_suppression': 0.2, 'average_precision': {'flower': 0.4008512353302792, 'hammer': 0, 'house': 0.8123436306340719, 'knife': 0.0818181818181818, 'person': 0.7726360331836617, 'rainbow': 0.0, 'rifle': 0.5921257391845627, 'rocket': 0.24586206896551724, 'scissors': 0, 'sun': 0.7486573576799141, 'tree': 0.13877314814814815}, 'map': 0.3448243086313033}]


In [21]:
from imageai.Detection.Custom import CustomObjectDetection
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath(
    "./final_model/models/detection_model-ex-001--loss-0031.882.h5")
detector.setJsonPath("./final_model/json/detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(
    input_image="47.jpg", output_image_path="test_res.jpg", minimum_percentage_probability=80)
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])


[341, 193, 424, 371]
[438, 208, 521, 357]
[441, 215, 517, 364]
[182, 230, 247, 431]
person  :  98.76635670661926  :  [341, 193, 424, 371]
person  :  97.50291109085083  :  [438, 208, 521, 357]
tree  :  82.70829319953918  :  [441, 215, 517, 364]
person  :  96.01204991340637  :  [182, 230, 247, 431]


In [155]:
#From the results of the object detecion model, we create a dataframe that contains the boxpoints of each object on the image (house, tree and person)
df = pd.DataFrame({'name': pd.Series(dtype='str'),
                   'xmin': pd.Series(dtype='int'),
                   'ymin': pd.Series(dtype='int'),
                   'xmax': pd.Series(dtype='int'),
                   'ymax': pd.Series(dtype='int'),
                   })
for detection in detections:
  new_row= pd.DataFrame()
  new_row = pd.DataFrame([[detection['name'],detection["box_points"][0],detection["box_points"][1],detection["box_points"][2],detection["box_points"][3]]],columns=['name','xmin','ymin','xmax','ymax'])
  df = pd.concat([df, new_row], ignore_index=True)
#We add three more columns that contains heights, widths and areas of the objects  
df['height'] = df['xmax']-df['xmin']
df['width'] = df['ymax']-df['ymin']
df['area'] = df['height']*df['width']

#We create another dataframe that contains the thresholds of height, width and area for each object, so we can make the interpretation  
thresholds = pd.DataFrame({'object': ['house', 'tree', 'person'], 'height': [(
    size[0]*30/100, size[0]*60/100), (size[0]*30/100, size[0]*80/100), ((size[0]*30/100, size[0]*40/100))], 'width': [(size[0]*30/100), (size[0]*5/100), (size[0]*5/100)], 'area': [(0), (0), (0)]})
thresholds.set_index('object',inplace=True)
# TO FILL ....

#After that, we create a function that takes as input the object that we want to analyse (measure the proportions), and then return the interpretation based on the height, width and area (for the moment, we only check the height of the objects) 
def interpretation(drawing,df) : #drawing is the object we want to check its proportions, and df is the dataframe that contains the informations of each object (for example: interpretation('house',df))
  interpretation = 'the size of the object'+' \"'+drawing +'\"'+' is normal'
  if drawing == 'tree' : 
    if int(df[df['name'] == drawing].height) < thresholds['height'][drawing][0] : 
      interpretation = 'The size of the tree in the drawing is very small, it relates to a weak ego'
    elif int(df[df['name'] == drawing].height) > thresholds['height'][drawing][1]:
      interpretation = 'The size of the tree in the drawing is very big, it relates to a strong inner stength and ego'
  
  elif drawing == 'house' : 
    if int(df[df['name'] == drawing].height) < thresholds['height'][drawing][0]:
      interpretation = 'The height of the house in the drawing is very small, it signifies rejection of the home and family life'
    elif int(df[df['name'] == drawing].height) > thresholds['height'][drawing][1]:
      interpretation = 'The height of the house in the drawing is very big, it signifies the view of the home as restrictive and controlling'
  
  elif drawing == 'person' : 
    if int(df[df['name'] == drawing].height) < thresholds['height'][drawing][0]:
      interpretation = 'The height of the person in the drawing is very small, it reflects feeling of inferiority and insignificance'
    elif int(df[df['name'] == drawing].height) > thresholds['height'][drawing][1]:
      interpretation = 'The height of the person in the drawing is extremly large, it tell us that the patient has a high-level of self esteem, or may suffer from delusions of grandeur'
      
  else : 
    return 'no drawing found'

  return interpretation #If interpretation == '', it means that we can tell anything about the sizes of the objects (the objects are in a normal size)



#After that, we crop the object we want to analyze on the image, using these two functions (the house object)
# FUNCTIONS 
def boxpoints_object(ob,df) : #df is the dataframe that contains the informations about each object, and ob is the object we want to crop (for the moment, we only analyze the house) 
  if ob not in ['house','tree','person'] :
    print('no object found')
  else : 
    coordinates_obj = df[df.name==ob][['xmin','ymin','xmax','ymax']]
    if coordinates_obj.shape[0] != 1 :
      print('Error, the model detected zero or more than one',ob,'object' )
      return 0
    else : 
      coordinates_obj=coordinates_obj.values.flatten().tolist()
      return [coordinates_obj[0], coordinates_obj[1], coordinates_obj[2], coordinates_obj[3]] 

#If the function returns 0, it means the presence of zero or more than one same object in the dataframe (for example: the dataset contains two house objects, or zero house object)

def return_cropped_image(im,boxpoints) : #this function takes as input the original image, and then return the cropped image using the boxpoints returned by 'boxpoints_object' function  
  im_cropped = im.crop((boxpoints_house))
  im_cropped = im_cropped.resize((512,512))
  return im_cropped

#The cropped image (of the house) is then fed to the next model for further analysis (hassan's model)
#ICI 


In [156]:
print(df)
print(interpretation('house', df))
print(interpretation('tree', df))
print(interpretation('person', df))


     name  xmin  ymin  xmax  ymax  height  width    area
0     sun     0     8   206   161     206    153   31518
1    tree   162    59   440   610     278    551  153178
2   house   482   214   856   601     374    387  144738
3  person   345   335   485   692     140    357   49980
the size of the object "house" is normal
the size of the object "tree" is normal
The height of the person in the drawing is very small, it reflects feeling of inferiority and insignificance
